In [1]:
import yfinance as yf
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Hisse senedi sembollerini belirleyin
stock_symbols = ['YKBNK.IS', "GARAN.IS",'AKBNK.IS', 'ISCTR.IS']

# Diğer faktör sembollerini belirleyin
factor_symbols = ['^VIX', 'USDTRY=X', 'GC=F', 'CL=F']  # Korku endeksi, Dolar/TL, Altın/TL, Petrol fiyatı

# Veri aralığı
start_date = '2018-08-10'
end_date = '2023-08-10'

# Hisse senedi ve faktör verilerini çekin
stock_data = yf.download(stock_symbols, period="5y")['Adj Close']
factor_data = yf.download(factor_symbols, period="5y")['Adj Close']

# Günlük getirileri hesaplayın
stock_returns = stock_data.pct_change().dropna()
factor_returns = factor_data.pct_change().dropna()

# Veri hazırlığı
all_data = stock_returns.join(factor_returns)
all_data.dropna(inplace=True)

# Bağımlı değişken olarak portföy getirileri, bağımsız değişkenler olarak faktör getirileri
X = all_data[factor_symbols].values
y = all_data[stock_symbols].mean(axis=1).values

# Modeli eğitin
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# Portföy getirilerini hesaplayın
portfolio_returns = model.predict(factor_returns)

# Parametrik VaR hesaplaması (normal dağılım varsayımı)
confidence_level = 0.99
mean_return = np.mean(portfolio_returns)
std_deviation = np.std(portfolio_returns)

z_score = np.percentile(np.random.normal(size=100000), (1 - confidence_level) * 100)
portfolio_var = mean_return + std_deviation * z_score

# Sonucu yazdırın
print(f"Portföy VaR ({confidence_level * 100}%): {portfolio_var:.6f}")


[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  4 of 4 completed
Portföy VaR (99.0%): -0.017200


c:\Users\mehme\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
